# Start and Test Rekognition Model

## 1- Start your Rekognition model (Editing required)

In [ ]:
!aws rekognition start-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:230950097573:project/ppe/version/ppe.2020-07-10T08.50.41/1594389041609" \
  --min-inference-units 1 \
  --region us-east-1

## 2- Define inference function (No editing required)

In [ ]:
confidence =60
import boto3
import io
%matplotlib inline
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

def show_custom_labels(model,bucket,photo):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=1,
        ProjectVersionArn=model)
   
   
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  

    colors = ('#FF0000','#0000FF','#FF00FF','#F1C40F')
    dict1 = {'earmuffs':0,'gloves':1,'goggle':2,'mask':3}  
    # calculate and display bounding boxes for each detected custom label       
    for customLabel in response['CustomLabels']:
        if customLabel['Confidence'] >= confidence:            
            if 'Geometry' in customLabel:
                box = customLabel['Geometry']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                draw.text((left + width-20,top-15), customLabel['Name'], fill= colors[dict1[customLabel['Name']]])
                draw.text((left,top-15), str(customLabel['Confidence'])[:6], fill= colors[dict1[customLabel['Name']]])#, font='Arial.ttf') 

                points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top))
                draw.line(points, fill= colors[dict1[customLabel['Name']]], width=5)

    image.show()
    display(image)
    image = image.convert('RGB')
    image.save('inference.jpg',optimize=True,quality=300)

    return len(response['CustomLabels'])

## 3- Detect Tools (Editing required)

In [ ]:
bucket="edo-buildout-images"
model= "arn:aws:rekognition:us-east-1:230950097573:project/ppe/version/ppe.2020-07-10T08.50.41/1594389041609"
photo = 'train1.png'


# Example:

# bucket = 'bucket-name'
# model = "arn:aws:rekognition:us-east-1:745137163025:project/cop/version/cop.2020-01-31T12.17.29/1580494650055"
# photo = 'pictures/4.png'
label_count=show_custom_labels(model,bucket,photo)

## 4- Stop your Rekognition model (Editing required).

In [ ]:
!aws rekognition stop-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:230950097573:project/project2/version/project2.2020-07-20T12.19.03/1595240343949" \
  --region us-east-1